In [1]:
import json
file_path = "intends.json"
with open(file_path, 'r') as file:
        topology = json.load(file)
print(topology)

{'AS1': {'protocol': 'RIP', 'routers': {'R1': {'R2': 'GigabitEthernet1/0'}, 'R2': {'R1': 'GigabitEthernet1/0'}}, 'neighbor': {}, 'address': '2001:db8:1::', 'subnet_mask': '/64'}}


In [2]:
import json
import sys
import time
from concurrent.futures import ThreadPoolExecutor

base_config = [
    "version 15.2",
    "service timestamps debug datetime msec",
    "service timestamps log datetime msec",
    "boot-start-marker",
    "boot-end-marker",
    "no aaa new-model",
    "no ip icmp rate-limit unreachable",
    "ip cef",
    "no ip domain lookup",
    "ipv6 unicast-routing",
    "ipv6 cef",
    "multilink bundle-name authenticated",
    "ip tcp synwait-time 5",
    "ip forward-protocol nd",
    "no ip http server",
    "no ip http secure-server",
    "control-plane",
    "line con 0",
    " exec-timeout 0 0",
    " privilege level 15",
    " logging synchronous",
    " stopbits 1",
    "line aux 0",
    " exec-timeout 0 0",
    " privilege level 15",
    " logging synchronous",
    " stopbits 1",
    "line vty 0 4",
    " login",
    "end"]


def main(topology):
    with ThreadPoolExecutor() as executor:
        futures = []
        subnet_dict = give_subnet_number(topology)
        subnet_interconnexion_dict = give_subnet_interconnexion(topology, subnet_dict)
        for AS in topology:
            for router in topology[AS]['routers']:
                future = executor.submit(process_router, router, AS, subnet_dict[AS], subnet_interconnexion_dict[AS], topology)
                futures.append(future)

        for future in futures:
            future.result()


def process_router(router, AS, subnet_dict, subnet_interconnexion_dict, topology):
    create_base_cfg(router, base_config)
    create_loopback_interface(router, topology[AS])
    create_router_interfaces(router, AS, topology[AS], subnet_dict)
    activate_protocols(router, AS, topology[AS], subnet_interconnexion_dict,subnet_dict)


def insert_cfg_line(router, index_line, data):
    # insert une information 'data' à la ligne 'line'.
    with open(f'i{router[1:]}_startup-config.cfg', 'r') as file:
        lines = file.readlines()
        lines.insert(index_line, data)
    with open(f'i{router[1:]}_startup-config.cfg', 'w') as file:
        file.writelines(lines)


def create_base_cfg(router, base_config):
    # créer pour un routeur, son fichier cfg de base à partir d'une liste 'base config', et insert l'hostname.
    with open(f'i{router[1:]}_startup-config.cfg', 'w') as file:
        for entry in base_config:
            file.write(entry + '\n')
    insert_cfg_line(router, 3, f"hostname {router}\n")


def find_index(router, line):
    # Trouve l'indice correcte pour inserer une nouvelle ligne juste après la ligne 'line'.
    index_line = 1
    with open(f'i{router[1:]}_startup-config.cfg', 'r') as file:
        lines = file.readline()
        while lines != line:
            lines = file.readline()
            index_line += 1
    return index_line


def create_router_interfaces(router, AS, as_topology, subnet_dict):
    # insert dans le cfg du routeur toutes les ses interfaces et leurs ipv6 correspondantes.
    index_line = find_index(router, line="ip tcp synwait-time 5\n")

    for neighbor in as_topology['routers'][router].keys():
        if router[1:] < neighbor[1:]:
            subnet_index = subnet_dict[(router, neighbor)]
            router_index = 1
        else:
            subnet_index = subnet_dict[(neighbor, router)]
            router_index = 2

        insert_cfg_line(router, index_line,
                        f"interface {as_topology['routers'][router][neighbor]}\n no ip address\n negotiation auto\n ipv6 address {as_topology['address']}{subnet_index}::{router_index}{as_topology['subnet_mask']}\n ipv6 enable\n")
        index_line += 5


def give_subnet_number(as_topology):  # Crée le tableau des subnets pour tout le json
    subnet_dict = dict()
    for AS in as_topology:
        subnet_dict[AS] = dict()
        subnet_number = 1
        for router in as_topology[AS]['routers']:
            for neighbor in as_topology[AS]['routers'][router]:
                if router[1:] < neighbor[1:]:
                    subnet_dict[AS][(router, neighbor)] = subnet_number
                    subnet_number += 1
    return subnet_dict

def last_entries_subnet(subnet_dict):
    last_entry = dict()
    for AS in subnet_dict:
        last_entry[AS] = list(subnet_dict[AS].values())[-1]
    return last_entry
def give_subnet_interconnexion(as_topology, subnet_dict):  # Crée le tableau des subnets des interconnexion entre AS pour tout le json
    subnet_interconnexion_dict = dict()
    last_entries = last_entries_subnet(subnet_dict)
    for AS in as_topology:
        subnet_interconnexion_dict[AS] = dict()

    for AS in as_topology:
        for AS_neighbor in as_topology[AS]['neighbor']:
            for router1 in as_topology[AS]['neighbor'][AS_neighbor]:
                for router2 in as_topology[AS]['neighbor'][AS_neighbor][router1]:
                    if (router1, router2) not in subnet_interconnexion_dict[AS].keys() and (router2, router1) not in subnet_interconnexion_dict[AS].keys():
                        if AS < AS_neighbor:
                            subnet_interconnexion_dict[AS][(router1, router2)] = str(int(AS[3:])) + str((last_entries[AS] + 1)) + "::1"
                            subnet_interconnexion_dict[AS_neighbor][(router2, router1)] = str(int(AS[3:])) + str((last_entries[AS] + 1)) + "::2"
                        else:
                            subnet_interconnexion_dict[AS][(router1, router2)] = str(int(AS_neighbor[3:])) + str((last_entries[AS] + 1)) + "::1"
                            subnet_interconnexion_dict[AS_neighbor][(router2, router1)] = str(int(AS_neighbor[3:])) + str((last_entries[AS] + 1)) + "::2"
                        last_entries[AS] += 1
    return subnet_interconnexion_dict


def is_rip(as_topology):
    # retourne True si RIP est à activer, False sinon.
    return True if as_topology["protocol"] == "RIP" else False


def is_ospf(as_topology):
    # retourne True si RIP est à activer, False sinon
    return True if as_topology["protocol"] == "OSPF" else False


def activate_protocols(router, AS, as_topology, subnet_interconnexion_dict,subnet_dict):
    # active tous les protocols d'un routeur.
    router_id = give_router_id(router)
    if is_ospf(as_topology):
        activate_ospf(router, as_topology, router_id)
    elif is_rip(as_topology):
        activate_rip(router, as_topology)
    activate_bgp(router, AS, as_topology, subnet_interconnexion_dict,subnet_dict)


def activate_ospf(router, as_topology, router_id):
    # active OSPF sur le routeur.
    index_line = find_index(router, "no ip http secure-server\n")
    insert_cfg_line(router, index_line, f"ipv6 router ospf 1\n router-id {router_id}\n")
    for interface in as_topology["routers"][router].values():
        index_line = find_index(router, f"interface {interface}\n") + 4
        insert_cfg_line(router, index_line, " ipv6 ospf 1 area 0\n")

    if is_border_routers(router, as_topology):

        index_line = find_index(router, "ip forward-protocol nd\n") - 1
        insert_cfg_line(router, index_line, "router ospf 1\n")
        index_line = find_index(router, f" router-id {router_id}\n")
        for AS_neighbor in as_topology["neighbor"]:
            if router in as_topology["neighbor"][AS_neighbor].keys():
                for interface in as_topology["neighbor"][AS_neighbor][router].values():
                    insert_cfg_line(router, index_line, f" passive-interface {interface}\n")
                    

def activate_rip(router, as_topology):
    # active RIP sur le routeur.
    rip_process_name = "process"
    index_line = find_index(router, "no ip http secure-server\n")
    insert_cfg_line(router, index_line, f"ipv6 router rip {rip_process_name}\n redistribute connected\n")
    for interface in as_topology["routers"][router].values():
        index_line = find_index(router, f"interface {interface}\n") + 4
        insert_cfg_line(router, index_line, f" ipv6 rip {rip_process_name} enable\n")


def give_router_id(router):
    x = router[1:]
    return f"{x}.{x}.{x}.{x}"


def get_subnet_interconnexion(subnet_interconnexion_dict, routeur1, routeur2):
    return subnet_interconnexion_dict[(routeur1, routeur2)] or subnet_interconnexion_dict[(routeur2, routeur1)]


def activate_bgp(routeur, AS, as_topology, subnet_interconnexion_dict, subnet_dict):
    
    index_line = find_index(routeur, "ip forward-protocol nd\n") - 1
    if is_border_routers(routeur, as_topology):

        for AS_neighbor in as_topology["neighbor"]:
            for neighborRouter in as_topology["neighbor"][AS_neighbor]:
                if neighborRouter == routeur:
                    for neighborRouter2 in as_topology["neighbor"][AS_neighbor][neighborRouter]:
                        insert_cfg_line(routeur, index_line,
                                        f"interface {as_topology['neighbor'][AS_neighbor][neighborRouter][neighborRouter2]}\n no ip address\n negotiation auto\n ipv6 address {as_topology['address'][:-1]}{get_subnet_interconnexion(subnet_interconnexion_dict, routeur, neighborRouter2)}{as_topology['subnet_mask']}\n ipv6 enable\n")
                        index_line += 5
                        if is_ospf(as_topology):
                            insert_cfg_line(routeur, index_line," ipv6 ospf 1 area 0\n")
                            index_line +=1
                                            
    insert_cfg_line(routeur, index_line,
                    f"router bgp 10{AS[3:]}\n bgp router-id {give_router_id(routeur)}\n bgp log-neighbor-changes\n no bgp default ipv4-unicast\n")
    index_line += 4
    #neighbor autre as
    for AS_neighbor in as_topology["neighbor"]:
        if is_border_routers(routeur, as_topology):
           
            for subnet in subnet_interconnexion_dict:
                if subnet[0] == routeur:
                    if int(AS[3:]) < int(AS_neighbor[3:]):
                        network_tail = subnet_interconnexion_dict[subnet][:-1]+"2"
                        
                    else:
                        network_tail = subnet_interconnexion_dict[subnet][:-1]+"1"
                        

            insert_cfg_line(routeur, index_line, f" neighbor {as_topology['address'][:-1]}{network_tail} remote-as 10{AS_neighbor[3:]}\n")
            index_line +=1

    neighborConf = ""
    index_sum = 0
    for router in as_topology["routers"]:
        if router != routeur:
            neighborConf += f" neighbor 2001::{router[1:]} remote-as 10{AS[3:]}\n neighbor 2001::{router[1:]} update-source Loopback0\n"
            index_sum += 2
    insert_cfg_line(routeur, index_line, neighborConf)
    index_line += index_sum
    insert_cfg_line(routeur, index_line, f" address-family ipv4\n exit-address-family\n address-family ipv6\n")
    index_line += 3
    neighborConf = ""
    index_sum = 0

    for router in as_topology["routers"]:
        if router != routeur:
            neighborConf += f"  neighbor 2001::{router[1:]} activate\n"
            index_sum += 1
    if is_border_routers(routeur, as_topology):
        insert_cfg_line(routeur, index_line, f"  neighbor {as_topology['address'][:-1]}{network_tail} activate\n")
        index_line +=1

    insert_cfg_line(routeur, index_line, neighborConf)
    index_line += index_sum
    if is_border_routers(routeur, as_topology):
        
        create_networks(as_topology,routeur, subnet_dict, index_line)
        index_line +=1
    insert_cfg_line(routeur, index_line, " exit-address-family\n")


def is_border_routers(router, as_topology):
    state = False
    for AS_neighbor in as_topology["neighbor"]:
        if router in as_topology["neighbor"][AS_neighbor].keys():
            state = True
    return state

def create_networks(as_topology, router, subnet_dict, index_line):
    
    for AS_neighbor in as_topology["neighbor"]:        
        for AS in as_topology["pingable"]:
            if AS == AS_neighbor:
                network = as_topology["pingable"][AS]
                insert_cfg_line(router, index_line, f"  network {as_topology['address']}{subnet_dict[network[0][0], network[0][1]]}::{as_topology['subnet_mask']}\n" )
                
                    

def create_loopback_interface(router, as_topology):
    index_line = find_index(router, line="ip tcp synwait-time 5\n")
    insert_cfg_line(router, index_line, f"interface Loopback0\n no ip address\n ipv6 address 2001::{router[1:]}/128\n")
    index_line += 3
    if is_ospf(as_topology):
        insert_cfg_line(router, index_line, " ipv6 ospf 1 area 0\n")


if __name__ == "__main__":
    start = time.time()
    with open("intends.json", 'r') as file:
        topology = json.load(file)
    main(topology)
    end = time.time()
    print(end - start)

0.018833637237548828


In [3]:
import os
import shutil
import time

def get_number_file(nom_fichier):
    if nom_fichier in correct_files:
        return int(nom_fichier[1:2])
    else:
        return None

def compare_and_paste(dossier_source, dossier_destination):
    noms_fichiers = os.listdir(os.path.join(os.getcwd(), dossier_destination))
    for nom_fichier in noms_fichiers:
        numero_fichier = get_number_file(nom_fichier)
        print(nom_fichier, numero_fichier)
        if numero_fichier is not None:
            chemin_fichier_source = os.path.join(dossier_source, nom_fichier)
            chemin_fichier_destination = os.path.join(dossier_destination, f"i{numero_fichier}_startupconfig.cfg")

            try:
                if os.path.exists(chemin_fichier_destination):
                    os.remove(chemin_fichier_destination)
                shutil.copy2(chemin_fichier_source, dossier_destination)
                print(f"Le fichier {nom_fichier} a été déplacé vers {dossier_destination}")
            except FileNotFoundError:
                print(f"Le fichier {nom_fichier} n'existe pas.")
            except PermissionError:
                print(f"Vous n'avez pas la permission de déplacer le fichier {nom_fichier}.")


start = time.time()

path_correct_files = os.path.join(os.getcwd())
correct_files = os.listdir(path_correct_files)

# Dossier GNS3
path_dossier_principal = r"C:\Users\User\GNS3\projects\2_routers\project-files\dynamips"
dossiers = os.listdir(path_dossier_principal)

for nom_dossier in dossiers:
    path_dossier_destination = os.path.join(path_dossier_principal, nom_dossier, "configs")
    if os.path.exists(path_dossier_destination):
        compare_and_paste(path_correct_files, path_dossier_destination)

end = time.time()
print("temps d'execution :", end-start)

i1_startup-config.cfg 1
Le fichier i1_startup-config.cfg a été déplacé vers C:\Users\User\GNS3\projects\2_routers\project-files\dynamips\32c26fe6-9500-432f-bd55-244449f44666\configs
i2_startup-config.cfg 2
Le fichier i2_startup-config.cfg a été déplacé vers C:\Users\User\GNS3\projects\2_routers\project-files\dynamips\87d86492-8d0f-4a7f-8e38-a3f5f9ff9af3\configs
temps d'execution : 0.003826618194580078


Without Threads

(to be tested)

In [ ]:
import json
import time

# Base configuration lines for the router
base_config = [
    "version 15.2",
    "service timestamps debug datetime msec",
    "service timestamps log datetime msec",
    "boot-start-marker",
    "boot-end-marker",
    "no aaa new-model",
    "no ip icmp rate-limit unreachable",
    "ip cef",
    "no ip domain lookup",
    "ipv6 unicast-routing",
    "ipv6 cef",
    "multilink bundle-name authenticated",
    "ip tcp synwait-time 5",
    "ip forward-protocol nd",
    "no ip http server",
    "no ip http secure-server",
    "control-plane",
    "line con 0",
    " exec-timeout 0 0",
    " privilege level 15",
    " logging synchronous",
    " stopbits 1",
    "line aux 0",
    " exec-timeout 0 0",
    " privilege level 15",
    " logging synchronous",
    " stopbits 1",
    "line vty 0 4",
    " login",
    "end"
]

def main(topology):
    """
    Main function to generate configurations for all routers in the topology.
    
    Inputs:
        topology (dict): JSON object containing the network topology.
    Outputs:
        None
    """
    # Generate subnet and interconnection data
    subnet_dict = give_subnet_number(topology)
    subnet_interconnexion_dict = give_subnet_interconnexion(topology, subnet_dict)

    # Process each router and generate configuration files
    for AS in topology:
        for router in topology[AS]['routers']:
            process_router(router, AS, subnet_dict[AS], subnet_interconnexion_dict[AS], topology)

def process_router(router, AS, subnet_dict, subnet_interconnexion_dict, topology):
    """
    Processes a router and generates its configuration file.
    
    Inputs:
        router (str): Router name.
        AS (str): Autonomous System ID.
        subnet_dict (dict): Subnet information for the AS.
        subnet_interconnexion_dict (dict): Interconnection subnet data.
        topology (dict): Network topology.
    Outputs:
        None
    """
    create_base_cfg(router, base_config)  # Create base configuration
    create_loopback_interface(router, topology[AS])  # Add loopback interface
    create_router_interfaces(router, AS, topology[AS], subnet_dict)  # Add interfaces
    activate_protocols(router, AS, topology[AS], subnet_interconnexion_dict, subnet_dict)  # Enable routing protocols

def insert_cfg_line(router, index_line, data):
    """
    Inserts a line of configuration into the router's configuration file at a specific index.
    
    Inputs:
        router (str): Router name.
        index_line (int): Line index to insert the data.
        data (str): Configuration line to insert.
    Outputs:
        None
    """
    # Read the current configuration file
    with open(f'i{router[1:]}_startup-config.cfg', 'r') as file:
        lines = file.readlines()
    
    # Insert the new line at the specified index
    lines.insert(index_line, data)

    # Write back the updated configuration
    with open(f'i{router[1:]}_startup-config.cfg', 'w') as file:
        file.writelines(lines)

def create_base_cfg(router, base_config):
    """
    Creates a base configuration file for a router.
    
    Inputs:
        router (str): Router name.
        base_config (list): List of base configuration lines.
    Outputs:
        None
    """
    with open(f'i{router[1:]}_startup-config.cfg', 'w') as file:
        for entry in base_config:
            file.write(entry + '\n')

    # Insert the hostname for the router
    insert_cfg_line(router, 3, f"hostname {router}\n")

def find_index(router, line):
    """
    Finds the index of a specific line in a router's configuration file.
    
    Inputs:
        router (str): Router name.
        line (str): Line to search for.
    Outputs:
        int: Index of the line.
    """
    with open(f'i{router[1:]}_startup-config.cfg', 'r') as file:
        lines = file.readlines()

    # Return the index of the target line
    for index, current_line in enumerate(lines):
        if current_line == line:
            return index
    return -1  # Return -1 if the line is not found

def create_router_interfaces(router, AS, as_topology, subnet_dict):
    """
    Adds all interfaces and their IPv6 addresses to a router's configuration file.
    
    Inputs:
        router (str): Router name.
        AS (str): Autonomous System ID.
        as_topology (dict): Topology of the AS.
        subnet_dict (dict): Subnet information.
    Outputs:
        None
    """
    index_line = find_index(router, line="ip tcp synwait-time 5\n")

    # Iterate over neighbors to configure interfaces
    for neighbor in as_topology['routers'][router].keys():
        if router[1:] < neighbor[1:]:
            subnet_index = subnet_dict[(router, neighbor)]
            router_index = 1
        else:
            subnet_index = subnet_dict[(neighbor, router)]
            router_index = 2

        insert_cfg_line(router, index_line,
                        f"interface {as_topology['routers'][router][neighbor]}\n no ip address\n negotiation auto\n ipv6 address {as_topology['address']}{subnet_index}::{router_index}{as_topology['subnet_mask']}\n ipv6 enable\n")
        index_line += 5

def give_subnet_number(as_topology):
    """
    Generates a dictionary of subnets for all routers in the topology.
    
    Inputs:
        as_topology (dict): Network topology.
    Outputs:
        dict: Dictionary of subnet information.
    """
    subnet_dict = {}
    for AS in as_topology:
        subnet_dict[AS] = {}
        subnet_number = 1
        for router in as_topology[AS]['routers']:
            for neighbor in as_topology[AS]['routers'][router]:
                if router[1:] < neighbor[1:]:
                    subnet_dict[AS][(router, neighbor)] = subnet_number
                    subnet_number += 1
    return subnet_dict

def give_subnet_interconnexion(as_topology, subnet_dict):
    """
    Generates subnet information for interconnections between Autonomous Systems (AS).
    
    Inputs:
        as_topology (dict): Network topology.
        subnet_dict (dict): Subnet information within AS.
    Outputs:
        dict: Interconnection subnet information.
    """
    subnet_interconnexion_dict = {}
    last_entries = last_entries_subnet(subnet_dict)
    for AS in as_topology:
        subnet_interconnexion_dict[AS] = {}

    # Generate inter-AS subnets
    for AS in as_topology:
        for AS_neighbor in as_topology[AS]['neighbor']:
            for router1 in as_topology[AS]['neighbor'][AS_neighbor]:
                for router2 in as_topology[AS]['neighbor'][AS_neighbor][router1]:
                    if (router1, router2) not in subnet_interconnexion_dict[AS] and (router2, router1) not in subnet_interconnexion_dict[AS]:
                        if AS < AS_neighbor:
                            subnet_interconnexion_dict[AS][(router1, router2)] = f"{int(AS[3:])}{last_entries[AS] + 1}::1"
                            subnet_interconnexion_dict[AS_neighbor][(router2, router1)] = f"{int(AS[3:])}{last_entries[AS] + 1}::2"
                        else:
                            subnet_interconnexion_dict[AS][(router1, router2)] = f"{int(AS_neighbor[3:])}{last_entries[AS] + 1}::1"
                            subnet_interconnexion_dict[AS_neighbor][(router2, router1)] = f"{int(AS_neighbor[3:])}{last_entries[AS] + 1}::2"
                        last_entries[AS] += 1
    return subnet_interconnexion_dict

def activate_protocols(router, AS, as_topology, subnet_interconnexion_dict, subnet_dict):
    """
    Activates routing protocols and configures their settings for the router.
    
    Inputs:
        router (str): Router name.
        AS (str): Autonomous System ID.
        as_topology (dict): Topology of the AS.
        subnet_interconnexion_dict (dict): Interconnection subnet data.
        subnet_dict (dict): Subnet information for the AS.
    Outputs:
        None
    """
    index_line = find_index(router, line="control-plane\n")
    
    # Enable RIPng if the AS is configured to use RIP
    if as_topology.get('protocol') == 'RIP':
        insert_cfg_line(router, index_line, "ipv6 router rip ripng\n")
        for neighbor in as_topology['routers'][router]:
            interface = as_topology['routers'][router][neighbor]
            insert_cfg_line(router, index_line + 1, f"interface {interface}\n ipv6 rip ripng enable\n")
            index_line += 2
    
    # Enable OSPFv3 if the AS is configured to use OSPF
    if as_topology.get('protocol') == 'OSPF':
        insert_cfg_line(router, index_line, f"ipv6 router ospf {AS}\n router-id {router[1:]}{router[1:]}\n")
        for neighbor in as_topology['routers'][router]:
            interface = as_topology['routers'][router][neighbor]
            insert_cfg_line(router, index_line + 1, f"interface {interface}\n ipv6 ospf {AS} area 0\n")
            index_line += 2
    
    # Configure BGP if the router participates in inter-AS communication
    if as_topology.get('bgp') == True:
        as_number = int(AS[3:])
        insert_cfg_line(router, index_line, f"router bgp {as_number}\n bgp router-id {router[1:]}{router[1:]}\n")
        for (local_router, remote_router), address in subnet_interconnexion_dict.items():
            if local_router == router:
                remote_as = as_topology['neighbor'][AS][local_router][remote_router]
                insert_cfg_line(router, index_line + 1, f"neighbor {address.split('::')[1]} remote-as {remote_as}\n")
                index_line += 1

def create_loopback_interface(router, as_topology):
    """
    Creates a loopback interface for the router.
    
    Inputs:
        router (str): Router name.
        as_topology (dict): Topology of the AS.
    Outputs:
        None
    """
    index_line = find_index(router, line="ip tcp synwait-time 5\n")
    loopback_ip = f"{as_topology['address']}0::{int(router[1:])}/128"
    insert_cfg_line(router, index_line, f"interface Loopback0\n ipv6 address {loopback_ip}\n ipv6 enable\n")


if __name__ == "__main__":
    start = time.time()
    with open("new_intends.json", 'r') as file:
        topology = json.load(file)
    main(topology)
    end = time.time()
    print(end - start)


Déplacement commenté

In [ ]:
import os
import shutil
import time

def get_number_file(nom_fichier):
    """
    Extracts a number from the filename if it exists in the correct_files list.
    The number is assumed to be at the second position (index 1:2) of the filename.

    Args:
        nom_fichier (str): The name of the file.

    Returns:
        int or None: Returns the extracted number if it is part of correct_files, 
                      otherwise None.
    """
    if nom_fichier in correct_files:
        return int(nom_fichier[1:2])  # Extract the number from the filename (index 1:2)
    else:
        return None

def compare_and_paste(dossier_source, dossier_destination):
    """
    Compares files in the source and destination directories. 
    If a valid file is found, it moves the file from source to destination with 
    a modified filename format.
    
    Args:
        dossier_source (str): Path to the source directory containing files to move.
        dossier_destination (str): Path to the destination directory where files 
                                    will be copied.
    """
    # List all files in the destination directory
    noms_fichiers = os.listdir(os.path.join(os.getcwd(), dossier_destination))
    
    for nom_fichier in noms_fichiers:
        # Try to extract a valid file number from the filename
        numero_fichier = get_number_file(nom_fichier)
        print(nom_fichier, numero_fichier)  # Print the filename and its number (if valid)
        
        # Proceed only if a valid number is found
        if numero_fichier is not None:
            chemin_fichier_source = os.path.join(dossier_source, nom_fichier)
            chemin_fichier_destination = os.path.join(dossier_destination, f"i{numero_fichier}_startupconfig.cfg")

            try:
                # Remove the existing destination file if it exists
                if os.path.exists(chemin_fichier_destination):
                    os.remove(chemin_fichier_destination)
                
                # Copy the source file to the destination directory
                shutil.copy2(chemin_fichier_source, dossier_destination)
                print(f"Le fichier {nom_fichier} a été déplacé vers {dossier_destination}")
            
            # Handle specific exceptions
            except FileNotFoundError:
                print(f"Le fichier {nom_fichier} n'existe pas.")  # File not found in source directory
            except PermissionError:
                print(f"Vous n'avez pas la permission de déplacer le fichier {nom_fichier}.")  # Insufficient permissions

# Record the start time of the script execution
start = time.time()

# Get the path to the current directory and list all the files in it
path_correct_files = os.path.join(os.getcwd())
correct_files = os.listdir(path_correct_files)  # List of files considered correct

# Define the main directory for GNS3 projects
path_dossier_principal = r"C:\Users\User\GNS3\projects\2_routers\project-files\dynamips"
dossiers = os.listdir(path_dossier_principal)  # List all subdirectories within the main GNS3 directory

# Iterate over all subdirectories in the main directory
for nom_dossier in dossiers:
    # Build the path to the 'configs' subdirectory within each project folder
    path_dossier_destination = os.path.join(path_dossier_principal, nom_dossier, "configs")
    
    # Check if the 'configs' directory exists within the project folder
    if os.path.exists(path_dossier_destination):
        # Call the function to compare and move files from source to destination
        compare_and_paste(path_correct_files, path_dossier_destination)

# Record the end time of the script execution and print the total execution time
end = time.time()
print("temps d'execution :", end-start)
